# 3D CAE Model Humerus Labelmap


In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from loguru import logger
import numpy as np
import time
import pickle
import h5py
from skimage.transform import downscale_local_mean,resize
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
import seaborn as sns
from sklearn.cluster import KMeans,DBSCAN

import tensorflow as tf
import keras
from keras import backend as K
from keras.layers import Conv3D,Conv3DTranspose,Dense,MaxPooling3D,UpSampling3D,Flatten,Dense,Reshape,GlobalAveragePooling3D,BatchNormalization,Dropout
from keras.activations import relu
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from itkwidgets import view

# My imports (se si modifica il file bisogna riavviare tutto il kernel)
from utilities import plot_all_slices_notzero,plot_slices,dice_coef

st=time.time()
with h5py.File("dataset_humerus_labelmap_res.hdf5","r") as f:
    humerus_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
st=time.time()
with h5py.File("dataset_humerus_labelmap_res_flipped.hdf5","r") as f:
    humerus_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")

In [ ]:
humerus_dataset.shape

## Downsampling

Meglio inziare con un downsampling per evitare problemi di memoria inizialmente.

Dataset: 288x210x393\
Padding a: 288x216x400\
Downsample per 4 a: 72x54x100

In [ ]:
humerus_dataset = np.pad(humerus_dataset,((0,0),(0,0),(3,3),(4,3)))

In [ ]:
humerus_dataset = resize(humerus_dataset,(humerus_dataset.shape[0],humerus_dataset.shape[1]/4,humerus_dataset.shape[2]/4,humerus_dataset.shape[3]/4),preserve_range=True,order=0,anti_aliasing=False)

In [ ]:
humerus_dataset.shape

In [ ]:
with h5py.File("resized_dataset_humerus_labelmap_res_flipped.hdf5","w") as f:
    f.create_dataset("mydataset",data=humerus_dataset)

# Model downsampled

Una volta importato il dataset possiamo iniziare ad occuparci della costruzione del modello. Partiamo da immagini downscalate

In [ ]:
st=time.time()
with h5py.File("resized_dataset_humerus_labelmap_res_flipped.hdf5","r") as f:
    humerus_dataset = f.get("mydataset")[:]
print(f"Tempo lettura dataset: {time.time()-st:.2f} sec")
humerus_dataset.shape

In [ ]:
resized_dataset_exp = np.expand_dims(humerus_dataset,axis=-1)
resized_dataset_exp.shape

In [ ]:
# Model downscaled con STRIDE (meglio)
embedding_dim = 128
inputs = keras.Input(shape=(resized_dataset_exp.shape[1],resized_dataset_exp.shape[2],resized_dataset_exp.shape[3],1))

# Encoder
x = Conv3D(filters=32,kernel_size=(3,3,3),padding="same",name="conv1_1")(inputs)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3D(filters=32,kernel_size=(3,3,3),strides=2,padding="same",name="conv1_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3D(filters=64,kernel_size=(3,4,3),padding="valid",name="conv2_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3D(filters=64,kernel_size=(3,3,3),strides=2,padding="same",name="conv2_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3D(filters=128,kernel_size=(4,3,3),padding="valid",name="conv3_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3D(filters=128,kernel_size=(3,3,3),strides=2,padding="same",name="conv3_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3D(filters=128,kernel_size=(3,3,3),padding="valid",name="conv4")(x)
x = BatchNormalization()(x)
x = relu(x)

# Bottleneck
shape_before_encoding = x.shape[1:]
x = Flatten(name="flatten")(x) 
flattened_size = x.shape[1]
x = Dropout(0.3)(x)
embedding = Dense(embedding_dim,activation="relu",name="embedding")(x)
x = Dropout(0.3)(embedding)
x = Dense(flattened_size,activation="relu",name="expanding")(x)
x = Reshape(shape_before_encoding,name="reshape")(x)

# Decoder
x = Conv3DTranspose(filters=128,kernel_size=(3,3,3),padding="valid",name="deconv4")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3DTranspose(filters=64,kernel_size=(3,3,3),padding="same",name="deconv3_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3DTranspose(filters=64,kernel_size=(3,3,3),strides=2,padding="same",name="deconv3_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3DTranspose(filters=32,kernel_size=(4,3,3),padding="valid",name="deconv2_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3DTranspose(filters=32,kernel_size=(3,3,3),strides=2,padding="same",name="deconv2_2")(x)
x = BatchNormalization()(x)
x = relu(x)

x = Conv3DTranspose(filters=16,kernel_size=(3,4,3),padding="valid",name="deconv1_1")(x)
x = BatchNormalization()(x)
x = relu(x)
x = Conv3DTranspose(filters=16,kernel_size=(3,3,3),strides=2,padding="same",name="deconv1_2")(x)
x = BatchNormalization()(x)
x = relu(x)

outputs = Conv3DTranspose(filters=1,kernel_size=(3,3,3),padding="same",name="deconv0",activation="sigmoid")(x)

model_downsample = keras.Model(inputs,outputs,name="model_downscaled")
model_downsample.summary()

In [ ]:
with open("training_history/hum_loss.pkl","rb") as f1,open("training_history/hum_val_loss.pkl","rb") as f2,open("training_history/hum_dice.pkl","rb") as f3,open("training_history/hum_val_dice.pkl","rb") as f4:
    tot_loss = pickle.load(f1)
    tot_val_loss = pickle.load(f2)
    tot_dice = pickle.load(f3)
    tot_val_dice = pickle.load(f4)

In [ ]:
tot_loss = []
tot_val_loss = []
tot_dice = []
tot_val_dice = []

In [ ]:
model_downsample.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[dice_coef]
)

checkpoint = ModelCheckpoint(
    "models_weights/humerus_flipped_checkpoint.weights.h5",
    monitor="loss",
    save_best_only=True,
    save_weights_only=True
)

history = model_downsample.fit(
    x=resized_dataset_exp,
    y=resized_dataset_exp,
    callbacks=[checkpoint],
    # validation_split=0.2,
    batch_size=16,
    epochs=50,
    verbose=1,
)

In [ ]:
model_downsample.save_weights("models_weights/humerus_flipped_loss0084_250ep_noval.weights.h5")

In [ ]:
# Non memorizzo le ultime 50 epoche senza validation 
tot_loss += history.history['loss'] # Per appendere liste tra loro
tot_val_loss+=history.history['val_loss']
tot_dice+=history.history['dice_coef']
tot_val_dice+=history.history['val_dice_coef']

In [ ]:
with open("training_history/hum_flip_loss.pkl","wb") as f1,open("training_history/hum_flip_val_loss.pkl","wb") as f2,open("training_history/hum_flip_dice.pkl","wb") as f3,open("training_history/hum_flip_val_dice.pkl","wb") as f4:
    pickle.dump(tot_loss,f1)
    pickle.dump(tot_val_loss,f2)
    pickle.dump(tot_dice,f3)
    pickle.dump(tot_val_dice,f4)

In [ ]:
# "Loss"
fig,ax = plt.subplots(2,1,figsize=(18,18))
start = 0
end = 200

# Loss
ax[0].plot(tot_loss[start:end+1],marker=".")
ax[0].plot(tot_val_loss[start:end+1],marker=".")
ax[0].set_title('Loss del Modello')
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epoche')
ax[0].set_xticks([x for x in range(0,end+1-start,5)])
ax[0].set_xticklabels([x for x in range(start,end+1,5)])
ax[0].legend(['Train', 'Validation'], loc='upper left')
ax[0].grid()

# Dice coefficient
ax[1].plot(tot_dice[start:end+1],marker=".")
ax[1].plot(tot_val_dice[start:end+1],marker=".")
ax[1].set_title('Dice Coefficient del Modello')
ax[1].set_ylabel('Dice Coefficient')
ax[1].set_xlabel('Epoche')
ax[1].set_xticks([x for x in range(0,end+1-start,5)])
ax[1].set_xticklabels([x for x in range(start,end+1,5)])
ax[1].legend(['Train', 'Validation'], loc='upper left')
ax[1].grid()

## Predictions

In [ ]:
model_downsample.load_weights("models_weights/humerus_flipped_loss0138_50ep.weights.h5")

In [ ]:
pred_ct = 400

In [ ]:
pred = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred_squeezed = np.squeeze(pred)
pred_squeezed_binary = np.where(pred_squeezed >= 0.5,1,0)

In [ ]:
dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred_squeezed)

In [ ]:
start_slice = 60
plot_slices(resized_dataset_exp[pred_ct],2,8,start_slice,title="Originale",titlesize=40)
plot_slices(pred_squeezed_binary,2,8,start_slice,title="Predizione Binaria",titlesize=40)
plot_slices(pred_squeezed,2,8,start_slice,title="Predizione",titlesize=40)

In [ ]:
# Carico i pesi 50 epoche
model_downsample.load_weights("models_weights/humerus_loss0106_100ep.weights.h5")
pred1 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred1_squeezed = np.squeeze(pred1)
pred1_squeezed_binary = np.where(pred1_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred1_squeezed))

In [ ]:
# Carico i pesi 150 epoche
model_downsample.load_weights("models_weights/humerus_loss0092_150ep.weights.h5")
pred2 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred2_squeezed = np.squeeze(pred2)
pred2_squeezed_binary = np.where(pred2_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred2_squeezed))

In [ ]:
# Carico i pesi 200 epoche
model_downsample.load_weights("models_weights/humerus_loss0085_200ep.weights.h5")
pred3 = model_downsample(np.expand_dims(resized_dataset_exp[pred_ct],axis=0))
pred3_squeezed = np.squeeze(pred3)
pred3_squeezed_binary = np.where(pred3_squeezed >= 0.5,1,0)
print(dice_coef(np.float32(resized_dataset_exp[pred_ct]),pred3_squeezed))

In [ ]:
start_slice = 60
plot_slices(resized_dataset_exp[pred_ct],1,8,start_slice,title="Originale",titlesize=40)
plot_slices(pred1_squeezed_binary,1,8,start_slice,title="Predizione dopo 100 epoche",titlesize=40)
plot_slices(pred2_squeezed_binary,1,8,start_slice,title="Predizione dopo 150 epoche",titlesize=40)
plot_slices(pred3_squeezed_binary,1,8,start_slice,title="Predizione dopo 200 epoche",titlesize=40)

## Feature extraction

In [ ]:
model_downsample.load_weights("models_weights/humerus_checkpoint.weights.h5") # Il checkpoint contiene quello con loss minima

In [ ]:
feature_model = keras.Model(model_downsample.inputs,model_downsample.get_layer("embedding").output)

In [ ]:
features = feature_model.predict(resized_dataset_exp)

In [ ]:
features.shape

In [ ]:
with open(f"processing/humerus_res_flip_features_250ep.pkl","wb") as f:
    pickle.dump(features,f)